# Library Data 読み込み

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold

import pprint

# CatBoostを利用した分類
import lightgbm as lgb

from sklearn.model_selection import GridSearchCV
# 指標を計算するため
from sklearn.metrics import accuracy_score, cohen_kappa_score, make_scorer, f1_score, recall_score
import re
from imblearn.pipeline import Pipeline
import lightgbm as lgb
from imblearn.over_sampling import SMOTE

#google drive読み込み
# from google.colab import drive, files
# drive.mount('/content/drive')
# dataPath = "/content/drive/My Drive/01 Lab/novel_bookmark/datasets/"
# test = pd.read_csv(dataPath + 'test.csv')
# train = pd.read_csv(dataPath +'train.csv')
# sub = pd.read_csv(dataPath +'sample_submission.csv')

#local 読み込み
dataPath = './datasets/'
titlePath = './data/titledata/'
keyPath = './data/keyworddata/'
storyPath = './data/storydata/'
allPath = './data/all/'

LenTrain = 40000

In [7]:
data = pd.read_csv("./tmp/tdidf_50.csv")

train = data[:LenTrain]
test = data[LenTrain:]
train = train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
train = train.fillna(0)
x_train = train.drop('fav_novel_cnt_bin', axis=1)#訓練データ
y_train = train['fav_novel_cnt_bin']#訓練データの答え
print(len(train))

40000


# Grid Search

In [8]:
skf = StratifiedKFold(
  n_splits=5,
  shuffle=True,
  random_state=0
)

# モデルの学習
model = Pipeline([
  ('sampling', SMOTE()),
  ('clf', lgb.LGBMClassifier())
])

# パラメーターを設定する
param_grid = {
  "clf__max_depth": [10, 25, 50, 75],
  "clf__learning_rate" : [0.001,0.01,0.05,0.1],
  "clf__num_leaves": [100,300,900,1200],
  # "clf__n_estimators": [100,200,500]
}
# パラメータチューニングをグリッドサーチで行うために設定する
## このGridSearchCV には注意が必要 scoring は そのスコアを基準にして最適化する
grid_result = GridSearchCV(
  estimator = model,
  param_grid = param_grid,
  scoring = 'balanced_accuracy',
  cv = skf,
  verbose=3,
  return_train_score = True,
  n_jobs = -1
)

grid_result.fit(x_train, y_train)

gs_result = pd.DataFrame.from_dict(grid_result.cv_results_)
gs_result.to_csv('./smallData/grid_result_tdidf.csv')

pprint.pprint(grid_result.best_estimator_)

Fitting 5 folds for each of 64 candidates, totalling 320 fits


In [ ]:
grid_result = pd.read_csv('./smallData/grid_result_0.csv', index_col=0)
grid_result = grid_result.sort_values('rank_test_score')
grid_result = grid_result[['rank_test_score', 'param_clf__num_leaves', 'param_clf__max_depth', 'param_clf__learning_rate', 'mean_test_score']]
grid_result.to_csv('./tmp/grid_result_0_process.csv')